In [4]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=67f0ceafdca61a4cd68fe109344e2d6f869a298cb83ae640b99f6f09069d74ad
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [73]:
from random import randint
def CreateRandomEdges(numberNodes):
  edges=[]
  for x in range(numberNodes):
    for y in range(numberNodes):
      if x==y:
        continue
      if randint(1,10) == 10:
        edges.append((x, y, randint(1,30)))
  return edges

In [68]:
def PrepararMensaje(oldShortestPath, edgesRDD):
  # print(oldShortestPath.collect())
  # print(edgesRDD.keys().collect())
  # print(edgesRDD.collect())
  joined = oldShortestPath.leftOuterJoin(edgesRDD,3)
  mapped = joined.values().map(lambda x: (x[1][0], x[0]+x[1][1]))
  seqFunc = (lambda x, y: (min(x,y)))
  combFunc = (lambda x, y: (min(x,y)))

  aggregate = mapped.aggregateByKey(30000, seqFunc, combFunc)
  nearlyThere = aggregate.leftOuterJoin(oldShortestPath)
  final = nearlyThere.mapValues(lambda x: min(x[0],x[1]))
  # print(sorted(final.collect()))
  return final

In [69]:
def UpdateShortestPath(oldShortestPath,agregate):
  pass

In [70]:
def GetShortestPath(initialShortestPath, edges):
  oldShortestPath = initialShortestPath
  for x in range(1000):
    newShortestPath = PrepararMensaje(oldShortestPath, edges)
    # newShortestPath = UpdateShortestPath(oldShortestPath,agregate)
    if sorted(oldShortestPath.collect()) == sorted(newShortestPath.collect()):
      break
    oldShortestPath = newShortestPath
  return newShortestPath

In [76]:
def FullShortestPath(nodes, edges, initialNode):
  nodesRDD = sc.parallelize(nodes)
  edgesRDD = sc.parallelize(map(lambda x: (x[0],(x[1],x[2])),edges))
  initialShortestPath = nodesRDD.map(lambda x: (x,len(nodes)*30) if x!=initialNode else (x,0))
  # print(initialShortestPath.collect())
  return GetShortestPath(initialShortestPath, edgesRDD)

In [78]:
edges = CreateRandomEdges(100)
# print(edges)
print(sorted(FullShortestPath([x for x in range(100)],edges,0).collect()))

[(0, 0), (1, 15), (2, 10), (3, 12), (4, 14), (5, 17), (6, 13), (7, 14), (8, 4), (9, 15), (10, 18), (11, 12), (12, 18), (13, 10), (14, 24), (15, 10), (16, 21), (17, 20), (18, 13), (19, 15), (20, 15), (21, 8), (22, 17), (23, 12), (24, 13), (25, 17), (26, 13), (27, 15), (28, 20), (29, 13), (30, 12), (31, 19), (32, 17), (33, 13), (34, 22), (35, 3), (36, 16), (37, 20), (38, 19), (39, 22), (40, 18), (41, 5), (42, 15), (43, 10), (44, 16), (45, 15), (46, 16), (47, 10), (48, 18), (49, 13), (50, 10), (51, 20), (52, 19), (53, 10), (54, 20), (55, 13), (56, 7), (57, 15), (58, 28), (59, 19), (60, 13), (61, 8), (62, 16), (63, 16), (64, 10), (65, 23), (66, 8), (67, 20), (68, 16), (69, 22), (70, 21), (71, 18), (72, 16), (73, 22), (74, 10), (75, 12), (76, 9), (77, 21), (78, 15), (79, 21), (80, 21), (81, 18), (82, 14), (83, 17), (84, 11), (85, 18), (86, 18), (87, 16), (88, 12), (89, 12), (90, 15), (91, 5), (92, 8), (93, 18), (94, 12), (95, 18), (96, 12), (97, 15), (98, 17), (99, 13)]
